In [1]:
import npaths
import numpy as np
import pickle

In [2]:
with open('power.pkl', 'rb') as f:
    power = pickle.load(f)

In [3]:
hyD = np.array(31*[5])
dz = np.array(3*[21]+12*[20]+3*[21])/100

In [5]:
core = npaths.MultiChannel(power,dz,hyD)

KeyError: "None of [Index(['hyD'], dtype='object')] are in the [columns]"